# Куда сходить покушать в Белгороде

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup



In [22]:
city = 'белгород'
type = 'еда'
url = f'https://yandex.ru/maps/213/moscow/search/{city}%20{type}/'

In [23]:

try:
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)
except Exception as e:
    print(f"An error occurred: {e}")


In [24]:
from IPython.display import clear_output 

count_places = 400  # Кол-во точек, которые хотим получить

elements = []

repeat_flag = 0

while len(elements) < count_places:
    clear_output(wait=True)
    time.sleep(1)
    
    len_before_scroll = len(elements)
    elements = driver.find_elements(By.CLASS_NAME, "search-business-snippet-view__content")
    
    driver.execute_script("arguments[0].scrollIntoView(true);", elements[-1])
    
    elements = driver.find_elements(By.CLASS_NAME, "search-business-snippet-view__content")
    len_after_scroll = len(elements)
    
    print(f'количество найденных локаций- {len(elements)}')
    
    if len_before_scroll == len_after_scroll:
        repeat_flag += 1
        if repeat_flag > 8:
            break
        

количество найденных локаций- 215


In [25]:
soup = BeautifulSoup(driver.page_source)

In [26]:
info_blocks = soup.find_all(class_='search-business-snippet-view__content')

In [27]:
info_list = {'name':[],'adress':[],'rait':[], 'rait_count':[], 
             'link':[],'closing_time':[], 'category': []
             }

for block in info_blocks:
    try:
        name = block.find(class_ = 'search-business-snippet-view__title')
        info_list['name'].append(name.text)
    except:
        info_list['name'].append('NaN')

    try:
        adress = block.find('a', 'search-business-snippet-view__address')
        info_list['adress'].append(adress.text)
    except:
        info_list['adress'].append('NaN')

    try:
        rait = block.find('span', 'business-rating-badge-view__rating-text')
        rait = rait.text.replace(",",".")
        info_list['rait'].append(rait)
    except:
        info_list['rait'].append('NaN')
    
    try:
        rait_count = block.find('span', 'business-rating-amount-view')
        if rait_count is None:
            rait_count = block.find('div', 'business-rating-with-text-view__count')
            info_list['rait_count'].append(rait_count.text)
        else:
            info_list['rait_count'].append(rait_count.text)
    except:
        info_list['rait_count'].append('NaN')

    try:
        closing_time = block.find('div', 'business-working-status-view')
        if closing_time:
            info_list['closing_time'].append(closing_time.text)
        else:
            info_list['closing_time'].append('NaN')
    except:
        info_list['closing_time'].append('NaN')
        
    try:
        category = block.find('a', 'search-business-snippet-view__category')
        if category:
            info_list['category'].append(category.text)
        else:
            info_list['category'].append('NaN')
    except:
        info_list['category'].append('NaN')


    try:
        link = block.find('a', 'search-business-snippet-view__rating')
        link = link['href'].replace('/reviews/','')
        info_list['link'].append(f'https://yandex.ru{link}')
    except:
        info_list['link'].append('NaN')
        
info_list

{'name': ['Farfor',
  'Перчини Grill&Wine',
  'Ниндзя ФУД',
  'Дао',
  'Тшм',
  'Белшашлык',
  'Шедевр',
  'Pitnica',
  'The doner',
  'The Doner',
  'KFC',
  'Mesto Grill',
  'Сидрерия',
  'Vasilchuki Chaihona № 1',
  'Бидон',
  'I am pho',
  'Футурист. bar+kitchen',
  'Пит-Стоп',
  'Грин фиш',
  'Мир шашлыка',
  'Папа Джонс',
  "Еда'ки",
  'Вкус Счастья',
  'KFC',
  'Потапыч',
  'ПАРА-пицца',
  'Корчма',
  'Дикая утка',
  'KFC Авто',
  'Потапыч',
  'MYBOX',
  'The GriLL',
  'Восточный смак',
  'Тук-Тук',
  'Восточное бистро',
  'Пицца 32 СМ',
  'Потапыч',
  'Стэйси пицца',
  'Еда в лаваше',
  'Лес и Лис',
  'Yellowcafe.ru',
  'Ale House',
  'Халяль',
  '12 Стульев',
  'Ресторан',
  'Пицаныч & Шавермыч',
  'Грильная 31',
  'Очень вкусная еда',
  'Сахар',
  'Трапеза',
  'Benny Kitchen',
  'Чучело',
  'Farfor',
  'Перчини Grill&Wine',
  'Старик Хинкалыч',
  'Потапыч',
  'Forest grill',
  'Башня',
  'Лабиринт',
  'Вкусно — и точка',
  'Смокерная',
  'Кенгуру',
  'Экватор',
  'Тшм',
  'Ар

In [31]:
df = pd.DataFrame(info_list)
df.head()

,name,adress,rait,rait_count,link,closing_time,category
0,Farfor,"Харьковский пер., 36Д, Белгород",4.3,391 оценка,https://yandex.ru/maps/org/farfor/73233893783,Открыто до 23:00,Доставка еды и обедов
1,Перчини Grill&Wine,"просп. Славы, 90А, Белгород",4.5,(2416),https://yandex.ru/maps/org/perchini_grill_wine...,Открыто до 23:00,Ресторан
2,Ниндзя ФУД,"ул. Костюкова, 39, Белгород",4.7,(651),https://yandex.ru/maps/org/nindzya_fud/1868725...,Открыто до 23:00,Кафе
3,Дао,"Харьковская ул., 3, Белгород, этаж 1",4.4,407 оценок,https://yandex.ru/maps/org/dao/17169128887,Открыто до 22:00,Ресторан
4,Тшм,"Белгородский просп., 93, Белгород",3.7,239 оценок,https://yandex.ru/maps/org/tshm/190737174920,Круглосуточно,Быстрое питание


In [30]:
df.shape

(215, 7)

In [32]:
df.describe()

,name,adress,rait,rait_count,link,closing_time,category
count,215,215,215,215,215,215,215
unique,171,195,21,186,211,28,20
top,MYBOX,"ул. Победы, 71, Белгород",4.3,153 оценки,NaN,Открыто до 23:00,Быстрое питание
freq,8,3,30,4,3,43,56


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          215 non-null    object
 1   adress        215 non-null    object
 2   rait          215 non-null    object
 3   rait_count    215 non-null    object
 4   link          215 non-null    object
 5   closing_time  215 non-null    object
 6   category      215 non-null    object
dtypes: object(7)
memory usage: 11.9+ KB


In [35]:
df.to_csv('data/belgorod_food_places.csv')